In [ ]:
!pip install category_encoders
!pip install nnetsauce
!pip install openml

In [3]:
import category_encoders as ce
import joblib
import nnetsauce as ns
import numpy as np
import openml
import pandas as pd
import warnings

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from time import time
from functools import lru_cache
from tqdm import tqdm

# 1. Utility functions and global variables
NROWS = 1000
NCOLS = 10

def select_NROWS_NCOLS(X, y):

    def replace_nan_with_median(arr):
        # Calculate the median of each column ignoring NaN values
        median_vals = np.nanmedian(arr, axis=0)

        # Iterate over each column index and replace NaN with the corresponding median value
        for col_idx in range(arr.shape[1]):
            col_values = arr[:, col_idx]
            nan_indices = np.isnan(col_values)
            if np.any(nan_indices):
                col_values[nan_indices] = median_vals[col_idx]

        return arr

    print(f"X.shape (initial): {X.shape}")
    print(f"y.shape (initial): {y.shape}")
    print("Encoding features and response...")

    # Label encoding and Hashing encoding
    encoder = ce.HashingEncoder(return_df=False)
    X = np.asarray(encoder.fit_transform(X, y)).astype(np.float32)
    X = replace_nan_with_median(X)

    print("...Done.")
    print(f"Finding top {NCOLS} features if necessary...")

    # Feature selection based on Random Forest feature importances
    if X.shape[1] > NCOLS:
        rf = RandomForestRegressor(n_estimators=50, random_state=42)
        rf.fit(X, y)
        indices = np.argsort(rf.feature_importances_)[::-1]
        top_cols = indices[:NCOLS]
        print(f"  top {NCOLS} indices: {top_cols}")
        X = X[:,top_cols]
        print(f"  X reduced shape: {X.shape}")
    print("...Done.")

    # Subsampling rows if necessary
    if X.shape[0] > NROWS:
        print(f"Subsampling to {NROWS} if necessary...")
        start = time()
        sub = ns.SubSampler(y=y.ravel(), n_samples=NROWS, seed=123, n_jobs=-1)
        idx_rows = sub.subsample().ravel()
        print(f"... Elapsed time for subsampling: {time() - start}")
        print("Number of rows in the subsample: ", len(idx_rows))
        return_X = replace_nan_with_median(X[idx_rows,:])
        return_y = y[idx_rows].ravel()
        print("Done.")
        return return_X, return_y
    else:
        return X, y

# 2. Get OpenML CC18 regression data
warnings.filterwarnings("ignore")

# Fetch the OpenML benchmark suite for regression tasks
benchmark_suite = openml.study.get_suite('OpenML-CC18')  # Obtain the benchmark suite
n_tasks = len(benchmark_suite.tasks)

Xys = {}

start = time()
for idx, task_id in tqdm(enumerate(benchmark_suite.tasks)):  # Iterate over all tasks
    try:
        print(f"\n\n Task #{idx}/{n_tasks} --------------------")
        warnings.filterwarnings("ignore")

        # Get the task information
        task = openml.tasks.get_task(task_id)  # Download the OpenML task
        # Get the data from OpenML
        task_name = task.get_dataset().name
        X_temp, y_temp = task.get_X_and_y()

        # Apply feature selection and subsampling
        Xys[task_name] = {"task_id": task_id,
                          "dataset": select_NROWS_NCOLS(X_temp, y_temp)}
        print(f"Elapsed time for task #{idx}: {time()-start}")
    except:
        continue

print(f"Total elapsed time: {time()-start}")


0it [00:00, ?it/s]



 Task #0/72 --------------------
X.shape (initial): (3196, 36)
y.shape (initial): (3196,)
Encoding features and response...
...Done.
Finding top 10 features if necessary...
  top 10 indices: [ 9 32 20 31 14  0  5 34 16 15]
  X reduced shape: (3196, 10)
...Done.
Subsampling to 1000 if necessary...


1it [00:05,  5.06s/it]

... Elapsed time for subsampling: 1.0208461284637451
Number of rows in the subsample:  999
Done.
Elapsed time for task #0: 5.063868284225464


 Task #1/72 --------------------
X.shape (initial): (20000, 16)
y.shape (initial): (20000,)
Encoding features and response...
...Done.
Finding top 10 features if necessary...


2it [00:13,  6.99s/it]

...

  top 10 indices: [   1 1243  351 1399    2  398    0 1229  508  246]
  X reduced shape: (3279, 10)
...Done.
Subsampling to 1000 if necessary...
... Elapsed time for subsampling: 0.023624897003173828
Number of rows in the subsample:  999
Done.
Elapsed time for task #69: 7877.581933021545


 Task #70/72 --------------------
X.shape (initial): (3186, 180)
y.shape (initial): (3186,)
Encoding features and response...
...Done.
Finding top 10 features if necessary...


71it [2:11:44, 395.55s/it]

  top 10 indices: [ 92 104  89  84  93  95  94  96  99  83]
  X reduced shape: (3186, 10)
...Done.
Subsampling to 1000 if necessary...
... Elapsed time for subsampling: 0.02333545684814453
Number of rows in the subsample:  999
Done.
Elapsed time for task #70: 7904.604424238205


 Task #71/72 --------------------
X.shape (initial): (5000, 20)
y.shape (initial): (5000,)
Encoding features and response...
...Done.
Finding top 10 features if necessary...


72it [2:11:53, 109.91s/it]

  top 10 indices: [ 7  9 19 17 10  4 12 18 16  6]
  X reduced shape: (5000, 10)
...Done.
Subsampling to 1000 if necessary...
... Elapsed time for subsampling: 0.02367687225341797
Number of rows in the subsample:  999
Done.
Elapsed time for task #71: 7913.815386295319
Total elapsed time: 7913.817942380905


In [4]:
import joblib

joblib.dump(Xys, "openml-Xys-2025-11-07.pkl")

['openml-Xys-2025-11-07.pkl']